### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

In [4]:
#import packages 

import os 
import requests  # Allows making HTTP requests to fetch data from websites or APIs
import json  # Helps in working with JSON data (storing, reading, and sending structured data)
from typing import List  # Provides type hints for lists, making code easier to understand
from dotenv import load_dotenv 
load_dotenv(override=True)  
from bs4 import BeautifulSoup 
from IPython.display import Markdown, display, update_display  # Used for displaying formatted outputs in Jupyter Notebooks
from openai import OpenAI  #interact with OpenAI's language models

In [7]:
#import the keys 
api_key = os.getenv("OPENAI_API_KEY")

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("Api key looks so far")
else:
    print("There is an issue with the api key")

Api key looks so far


In [11]:
#instantiate the model 
MODEL="gpt-4o-mini"
openai=OpenAI()

In [ ]:
# A class to represent a Webpage

# Some websites require a proper header to allow scraping
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# Create the class for the website 

"""
A Utility class to represent a website that we have scraped, now with links
"""

class Website:
    def __init__(self, url):
        self.url = url  # Store the website URL
        response = requests.get(url, headers=headers)  # Fetch webpage content
        self.body = response.content  # Store raw webpage content
        soup = BeautifulSoup(self.body, 'html.parser')  # Parse HTML

        # Extract the title, or set a default message if missing
        self.title = soup.title.string if soup.title else "No Title found"

        # Remove unnecessary elements and extract text
        if soup.body:
            for irrelavant in soup.body(["script", "style", "img", "input"]):
                irrelavant.decompose()  # Remove unwanted elements
            self.text = soup.body.get_text(separator="\n", strip=True)  # Get clean text
        else:
            self.text = ""  # Set text as empty if no body tag

        # Extract all links from the webpage
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]  # Filter out empty links

    def get_contents(self):
        # Return formatted webpage title and content
        return f"Webpage Title:\n{self.title}\nWebpage Content:\n{self.text}\n\n"



## lets test the function

In [23]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

In [24]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())

Webpage Title:
Home - Edward Donner
Webpage Content:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers an

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  


##### We will use "one shot prompting" i.e give llm an example

In [ ]:
#design the system prompt 

link_system_prompt="You are provided with a list of links found on a webpage.\
     You are able to decide which one of the links would be more relavant to include in a brochure about the company \
     such as links to an About Page, or a Company page or Career/Jobs page.\n "

link_system_prompt += "You should respond JSON as in this example:" 
link_system_prompt += """
{
    "links":[
        {"type":"about page", "url":"https://full.url/goes/here/about"}
        {"type":"careers page", "url":"https://another.full.url/careers"}
    
    ]
}

"""

In [27]:
print(link_system_prompt)

You are provided with a list of links found on a webpage.     You are able to decide which one of the links would be more relavant to include in a brochure about the company      such as links to be an About Page, or a Company page or Career/Jobs page.
 You should respond JSON as in this example:
{
    "links":[
        {"type":"about page", "url":"https://full.url/goes/here/about"}
        {"type":"careers page", "url":"https://another.full.url/careers"}

    ]
}




In [28]:
#design user prompt format of display from LLM

def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"

    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt
    

In [29]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [ ]:
# Now let's call the LLM (Language Model)

def get_links(url):
    website = Website(url)  # Create a Website object to scrape the given URL
    
    response = openai.chat.completions.create(  # Send a request to the AI model
        model=MODEL,  # Specify the AI model to use
        messages=[  # Provide system and user messages for context
            {"role": "system", "content": link_system_prompt},  # System instruction
            {"role": "user", "content": get_links_user_prompt(website)}  # User input with website data
        ],
        response_format={"type": "json_object"}  # Expect JSON-formatted output
    )

    result = response.choices[0].message.content  # Extract the AI-generated response
    return json.loads(result)  # Convert JSON string to a Python dictionary


In [32]:
#lets pull links for hugging face 
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'}]}

* as its visible we have only pulled only those links from hugging face which we will need to create the company brochure.
* lets once try and see what all links we left as it was not needed

In [35]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/sesame/csm-1b',
 '/google/gemma-3-27b-it',
 '/Qwen/QwQ-32B',
 '/deepseek-ai/DeepSeek-R1',
 '/RekaAI/reka-flash-3',
 '/models',
 '/spaces/sesame/csm-1b',
 '/spaces/Qwen/QwQ-32B-Demo',
 '/spaces/Wan-AI/Wan2.1',
 '/spaces/ASLP-lab/DiffRhythm',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets/open-r1/codeforces-cots',
 '/datasets/facebook/natural_reasoning',
 '/datasets/Congliu/Chinese-DeepSeek-R1-Distill-data-110k',
 '/datasets/open-r1/OpenR1-Math-220k',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffuser

* As its clearly visible we did need some many link to complete our task at hand.

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [ ]:
def get_all_details(url):
    result = "Landing Page:\n"  # Start with a label for the main page content
    result += Website(url).get_contents()  # Get and append the main page content

    # Call the function that interacts with the LLM 
    links = get_links(url)
    print("Found Links:", links)  # Print the extracted links for reference

    #now iterate through the links and pull their contents for that we will use class"Website"
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"  # Add link type for clarity
        result += Website(link["url"]).get_contents()  # Fetch and append the content of each linked page
    
    return result  # Return the full compiled webpage content


In [38]:
print(get_all_details("https://huggingface.co"))

Find Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing Page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Content:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
sesame/csm-1b
Updated
1 day ago
•
2.84k
•
1.04k
google/gemma-3-27b-it
Updated
5 days ago
•
2

* Call the LLM GPT4-o and this time we give it prompt template that instructs the gpt to articulately design the brochure with all the o/p we have gotten from "get_all_details" function

In [41]:
system_prompt="You are an assistant that analyzes the contents of several relavant pages from a compay website \
and creates a short brochure about the company from prospective customers, investors and recruits. Respond in \
markdown. Include details of company culture, customers and careers/jobs if you have the information."

In [42]:
print(system_prompt)

You are an assistant that analyzes the contents of several relavant pages from a compay website and creates a short brochure about the company from prospective customers, investors and recruits. Respond in markdown. Include details of company culture, customers and careers/jobs if you have the information.


In [47]:
def get_brochure_user_prompt(company_name, url):
    user_prompt= f"You are looking at a company called {company_name}\n"
    user_prompt+= f"Here are the contents of its landing page and other relevant pages;Use this information to build a short brochure of the company in markdown.\n"
    
    #call the fuction "get_all_details" for all info extraction
    user_prompt+=get_all_details(url)

    #put a threshold of no. of characters
    user_prompt+= user_prompt[:5_000] # truncate if more that 5,000 characters

    return user_prompt



In [48]:
get_brochure_user_prompt("Hugging Face","https://huggingface.co")

Find Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'company page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


'You are looking at a company called Hugging Face\nHere are the contents of its landing page and other relevant pages;Use this information to build a short brochure of the company in markdown.\nLanding Page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Content:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nsesame/csm-1b\nUpdated\n1 day ago\n•\n2.84k\n•\n1.04k\ngoogle/gemma-3-27b-it\nUpdated\n5 days ago\n•\n241k\n•\n723\nQwen/QwQ-32B\nUpdated\n6 days ago\n•\n417k\n•\n2.3k\ndeepseek-ai/DeepSeek-R1\nUpdated\n22 days ago\n•\n1.9M\n•\n11.4k\nRekaAI/reka-flash-3\nUpdated\n4 days ago\n•\n3.01k\n•\n281\nBrowse 1M+ models\nSpaces\nRunning\non\nZero\n422\n422\nSesame CSM\n🌱\nConversational speech generation

In [49]:
# call the LLM

def create_brochure(company_name, url):
    response=openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role":"system","content":system_prompt},
            {"role":"user","content":get_brochure_user_prompt(company_name, url)}
        ],
    )
    result=response.choices[0].message.content
    display(Markdown(result))

In [51]:
create_brochure("Hugging Face","https://huggingface.co")

Find Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is a leading AI community dedicated to building the future through advanced machine learning tools and collaboration. Our platform serves millions of developers, researchers, and enterprises globally, enabling them to create, share, and deploy machine learning models, datasets, and applications. 

## What We Offer
### **Collaboration Platform for AI**
- **Explore AI Applications**: Browse over **1 million models** and **250k datasets** for diverse machine learning tasks.
- **Community-Driven**: Participate in a vibrant ecosystem that encourages contributions to shared projects and models, ensuring rapid advancements in AI technology.

### **Advanced Tools for Developers**
- **State-of-the-Art Libraries**: Access to premier libraries such as [Transformers](https://huggingface.co/docs/transformers), [Diffusers](https://huggingface.co/docs/diffusers), and [Tokenizers](https://huggingface.co/docs/tokenizers) that facilitate the development of high-performance models.
- **Inference Endpoints & Spaces**: Deploy models easily on our optimized infrastructure, including GPU options, to meet various resource needs.

## Our Culture
At Hugging Face, we are on a mission to **democratize good machine learning**, encouraging collaboration, openness, and innovation. We believe in fostering a culture that supports creativity, inclusivity, and continuous learning. Our team is composed of passionate individuals from diverse backgrounds who work together to advance the field of machine learning while promoting ethical practices in AI development.

## Join Us
### Career Opportunities
We are always looking for talented individuals to join our diverse team. Whether you're an engineer, researcher, or business professional, we have positions that cater to various skill sets. Discover exciting openings on our [Careers Page](https://huggingface.co/jobs).

### Why Work Here?
- **Innovation-Driven Environment**: Engage with cutting-edge technology and contribute to groundbreaking AI projects.
- **Remote-Friendly**: Enjoy flexible work arrangements that support a healthy work-life balance.
- **Community Engagement**: Participate in outreach and collaboration opportunities that enhance your professional network.

## Who We Serve
Our diverse user base includes:
- **50,000+ Organizations**: From prominent companies like **Google**, **Amazon**, **Microsoft**, and many others, our enterprise solutions bolster their AI strategies.
- **Independent Developers and Researchers**: Explore, build, and share models and datasets with our cutting-edge tools to enable AI innovation on individual and community levels.

## Contact Us
For press inquiries, partnerships, or more information about our offerings, visit our [Contact Page](https://huggingface.co/contact).

---

**Hugging Face** – Your partner in building the future of AI!  
Join us in shaping the next wave of machine learning advancements.  
Explore our [Website](https://huggingface.co) today!

# Finally - A minor Improvement 

In [ ]:
# lets try creating this markdown in typwriter kind of fashion how chat gpt does

def stream_brochure(company_name,url):
    stream=openai.chat.completions.create(
    model=MODEL,
    messages=[
        {"role":"system","content":system_prompt},
        {"role":"user","content":get_brochure_user_prompt(company_name,url)}
    ],
    stream=True
)
    response=""
    display_handle=display(Markdown(""),display_id=True)

    #simpler way of displaying markdown 
    for chunk in stream:
        print(chunk.choices[0].delta.content or '')

    #just rewriting all the chunks again only for better display as markdown can be fuzzy as seen above
    for chunk in stream:
        response+=chunk.choices[0].delta.content or ''
        response=response.replace("```","").replace("markdown","")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [55]:
stream_brochure("Hugging Face","https://huggingface.co")

Find Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

### Welcome to Hugging Face – The AI Community Building the Future

Hugging Face is more than just an AI technology company; it's a thriving community of machine learning enthusiasts dedicated to the democratization of artificial intelligence. Established in 2016 and based in Paris and New York, we bring together innovators, researchers, and developers to collaborate, create, and share knowledge in the field of machine learning.

---

### Company Overview

- **Mission:** To democratize good machine learning, one commit at a time.
- **Core Values:** Community, collaboration, and open-source innovation.
- **Team:** A diverse group of over 200 members passionate about AI, including data scientists, engineers, and product managers.

---

### Key Products & Services

#### Hugging Face Hub

- A platform that hosts **1M+ models**, **250k+ datasets**, and **400k+ applications**.
- Collaborate on machine learning with unlimited public models and datasets.
- Explore all modalities: text, image, video, audio, and 3D.

#### Enterprise Solutions

- **Enterprise Hub:** Tailored for organizations needing secure and scalable AI solutions.
  - Starting at **$20/user/month**
  - Features include:
    - Single Sign-On (SSO)
    - Detailed audit logs for compliance
    - Priority support and dedicated resources

---

### Company Culture

At Hugging Face, we foster a culture of openness, collaboration, and continuous learning. We believe in the power of community, where everyone can contribute to the collective growth of technology and knowledge. Managed transparency and support from peers encourages innovation and experimentation.

- **Team Engagement:** Employees are invited to engage in community events, collaborate on projects, and run workshops.
- **Work-Life Balance:** We celebrate flexibility and the mental well-being of our employees.
  
---

### Our Customers

More than **50,000 organizations** utilize Hugging Face tools to empower their AI initiatives, including:

- **Major Tech Companies:** Google, Microsoft, Amazon, Intel
- **Non-Profit Organizations:** Ai2, contributing significantly to AI research and development.
  
---

### Career Opportunities

Hugging Face is continually looking for talent! We're actively recruiting for diverse roles ranging from engineers to community managers. Join us and be part of a passionate team shaping the future of AI.

**Current Openings:**  
- Machine Learning Engineers  
- Data Scientists  
- AI Researchers  
- Product Managers  

Explore our job listings on our [Careers Page](https://huggingface.co/jobs).

---

### Get in Touch

- **Website:** [huggingface.co](https://huggingface.co)  
- **Twitter:** [@huggingface](https://twitter.com/huggingface)   
- **GitHub:** [Hugging Face on GitHub](https://github.com/huggingface)  
- **LinkedIn:** [Hugging Face on LinkedIn](https://linkedin.com/company/hugging-face)  

Join us in our mission to build the future of artificial intelligence together!

--- 

*This brochure was created to provide an overview of Hugging Face for prospective customers, investors, and recruits.*

# System prompts are super import. Inside the system Prompt you set the tone of the LLM should use. 

In [56]:
#lets try and do the same but this time we will be creating a funny tone brochure

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [57]:
stream_brochure("Hugging Face","https://huggingface.co")

Find Links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company social link', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Welcome to Hugging Face! 🤗

*Where AI dreams come true, models collaborate, and datasets go to party!*

---

## Who We Are 🦸‍♀️

At **Hugging Face**, we believe in making the world a better place, one AI model at a time! Founded in 2016 and located in the picturesque setting of Paris, we’re the friendly neighborhood AI community building the future—because why not, right?

---

## What We Do 🚀

- **1M+ Models**: Browse an impressive collection of the latest models—yes, we counted! Just don’t try to hug them; they might get a little shy.
- **Datasets Galore**: Need data to train that genius AI of yours? We've got datasets covering every genre and flavor imaginable, from healthcare to entertainment. 🍿
- **Spaces for Everyone**: Create, share, and discover amazing applications and demos. These spaces are so great, they make a galaxy look like a slum!

---

## Our Take on Team Culture 🤗

At Hugging Face, we’re all about community and collaboration. Our team is a funky mix of innovators, nerds, and visionaries who spend more time hugging (online, of course) than not! We believe in democratizing machine learning, which means everyone gets a chance to shine, even if you’ve only made it to the garden level of AI knowledge. 🌱

---

## Customers Who Love Us 💖

Get ready to be starstruck! Over **50,000 organizations** join hands with us, including:
- Amazon 
- Google 
- Microsoft 
- Meta (AI edition)
  
You could say we’re kind of a big deal (and it’s not just the tinfoil hats)!

---

## Join Our Quest! 🏰

Looking for your next great adventure? We’re hiring! Whether you're an AI wizard, a data whisperer, or a code conjurer, there’s space for you! And yes, you’ll get to hug data...figuratively, of course.

### Open Positions:
- Machine Learning Engineers 🧙‍♂️
- Software Developers 💻
- Data Scientists 📊
- Technical Writers ✍️
  
---

## Why Choose Hugging Face? 🌈

- **Easy Pricing**: Starting at **$0.60/hour for GPU**! Seriously, don’t worry—we won't break the bank.
- **Enterprise Solutions**: Need security? We’ve got your back with enterprise-grade security for the big leagues. Talk about feeling safe and sound! 💼
- **Community Support**: We have forums where ideas flourish like mushrooms after a rainstorm! Join the discussions and share your thoughts.

---

## Connect with Us!

Check us out on social media, drop by the forums, or just send a friendly carrier pigeon! 🕊️ 

- [Twitter](https://twitter.com/huggingface)
- [GitHub](https://github.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [Discord](https://discord.gg/huggingface) *(Yes, that’s right, we have our own chat party!)*

---

### Ready to Dive In?

Whether you’re here to explore, contribute, or just hug some models, **Hugging Face** is where your AI journey begins. So, let’s get hugging! 

---

*Disclaimer: Hugging is strictly metaphorical when it comes to AI models. Please avoid actual hugging—especially for large models that might require their own bouncers!* 😄